In [1]:
# =========================================
# 0) Kurulum
# =========================================
!pip install -q unsloth rouge-score pandas peft

import os, re, shutil, torch, pandas as pd
from unsloth import FastModel
from rouge_score import rouge_scorer
from IPython.display import display
from tqdm.auto import tqdm

# =========================================
# 1) Google Drive
# =========================================
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 134.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 57.6 MB/s eta 0:0

In [2]:
# 2) Yollar — DÜZENLEYİN
# =========================================
# Test seti (JSONL): satır başına {"prompt": "...", "reference": "..."} veya {"input": "...", "output": "..."}
TEST_JSONL_PATH = "/content/drive/MyDrive/orcik/test_data.jsonl"

# Base IT model (HF id veya Drive yolu). İnternetsiz çalışacaksan Drive yolunu yaz.
BASE_IT_ID   = "google/gemma-3-1b-it"

# Merged (LoRA fused) ve Full-Finetune (tam eğitilmiş) model klasörleri (Drive)
LORA_PATH  = "/content/drive/MyDrive/orcik/model1B"
FULLFT_PATH  = "/content/drive/MyDrive/orcik/model4BFull_finetuned"

# Sonuç klasörü (Drive)
RESULTS_DIR  = "/content/drive/MyDrive/orcik"
os.makedirs(RESULTS_DIR, exist_ok=True)

In [3]:
# 3) Test setini yükle (Drive -> JSONL)
# =========================================
assert os.path.exists(TEST_JSONL_PATH), f"JSONL bulunamadı: {TEST_JSONL_PATH}"
df = pd.read_json(TEST_JSONL_PATH, lines=True)

# Esnek kolon isimleri
if {"prompt","reference"}.issubset(df.columns):
    df = df[["prompt","reference"]]
elif {"input","output"}.issubset(df.columns):
    df = df.rename(columns={"input":"prompt","output":"reference"})[["prompt","reference"]]
else:
    raise ValueError("JSONL satırları 'prompt/reference' veya 'input/output' alanlarını içermeli.")

df = df.astype(str).reset_index(drop=True)
df_azaltılan = df.sample(n=500, random_state=42)


# Drive içinde istediğin klasöre kaydet
save_path = "/content/drive/MyDrive/orcik/val_data.jsonl"
df_azaltılan.to_json(save_path, orient="records", lines=True, force_ascii=False)

print(f"JSONL dosyası Drive'a kaydedildi: {save_path}")
TEST_JSONL_PATH= "/content/drive/MyDrive/orcik/val_data.jsonl"

JSONL dosyası Drive'a kaydedildi: /content/drive/MyDrive/orcik/val_data.jsonl


In [4]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
basemodel, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LORA_PATH, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    load_in_4bit = False,
)
basemodel = FastLanguageModel.for_inference(basemodel) # Enable native 2x faster inference
text_streamer = TextStreamer(tokenizer)

==((====))==  Unsloth 2025.8.4: Fast Gemma3 patching. Transformers: 4.55.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

# Task
Generate completions for the `df_azaltılan` dataset using the `generate_response` function and save the results to a JSONL file named "results.jsonl" in the "results" directory.

## Define generation function

### Subtask:
Correct the `generate_response` function to properly format the input for the model and handle the output.


**Reasoning**:
The current `generate_response` function has a syntax error and does not properly format the input for the model or handle the output. I need to fix the function to accept a prompt, format it as a message list for the chat template, tokenize it, generate the response, decode the response, and return it.



In [5]:
def generate_response(prompt: str) -> str:
    messages = [{
        "role": "user",
        "content": [{"type": "text", "text": prompt}]
    }]
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False # Ensure it returns a string
    )
    inputs = tokenizer([text], return_tensors="pt").to("cuda")

    # Generate response, returning the output tensor instead of streaming
    outputs = basemodel.generate(
        **inputs,
        max_new_tokens=2048,
        pad_token_id=tokenizer.eos_token_id # Handle padding
    )

    # Decode the generated output and remove the prompt
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # The generated_text will include the prompt, we need to remove it
    # Find the start of the actual response after the prompt
    # This pattern might need adjustment based on the exact chat template output
    response_start_marker = "[/INST]"
    if response_start_marker in generated_text:
        response = generated_text.split(response_start_marker, 1)[1].strip()
    else:
        # Fallback if the marker is not found, might need further refinement
        response = generated_text.strip()

    return response

## Generate completions

### Subtask:
Iterate through the `df_azaltılan` DataFrame and generate a response for each prompt using the corrected `generate_response` function. Store the prompt, reference, and generated response in a new DataFrame.


**Reasoning**:
Iterate through the dataframe, generate responses, and store the results in a new dataframe as instructed.



In [6]:
results_list = []

# Using tqdm for a progress bar to monitor the generation process
for index, row in tqdm(df_azaltılan.iterrows(), total=df_azaltılan.shape[0]):
    prompt = row['prompt']
    reference = row['reference']
    generated_response = generate_response(prompt)

    results_list.append({
        'prompt': prompt,
        'reference': reference,
        'generated_response': generated_response
    })

results_df = pd.DataFrame(results_list)

display(results_df.head())

  0%|          | 0/500 [00:00<?, ?it/s]

,prompt,reference,generated_response
0,"Acil! Çocuğum ütüye dokundu, pijaması bacaklar...",Panik yapma! Pijamayı çekerek çıkarmaya kalkma...,"user\nAcil! Çocuğum ütüye dokundu, pijaması ba..."
1,"Parkta çocuk bisikletten düştü, nefes almayı k...",BAŞ-BOYUN TRAVMASINDAN ŞÜPHELENİYORSANIZ POZİS...,"user\nParkta çocuk bisikletten düştü, nefes al..."
2,"Oğlum futbol oynarken düştü, bileği aniden şiş...","Derhal hareket ettirmeyin! Bileği sabitleyin, ...","user\nOğlum futbol oynarken düştü, bileği anid..."
3,Terleme neden önemlidir?,"Terleme, vücut ısısını sabit tutmak için kriti...",user\nTerleme neden önemlidir?\nmodel\nTerleme...
4,"Bebeğim yemek yerken boğuldu, bilinci açık ama...",Derhal çene kaldırma manevrası yapın! Başını h...,"user\nBebeğim yemek yerken boğuldu, bilinci aç..."


In [7]:
results_df.to_json(os.path.join(RESULTS_DIR, "1bLORAresults.jsonl"), orient="records", lines=True)